<a href="https://colab.research.google.com/github/priyasin01/DEEP_LEARNING_MODELS/blob/master/Resnet_in_pyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import time
import numpy as np



# The code below is derived from yunjey/pytorch-tutorial/tutorials/02-intermediate/deep_residual_network

# ---------------------------------------------------------------------------- #
# An implementation of https://arxiv.org/pdf/1512.03385.pdf                    #
# See section 4.2 for the model architecture on CIFAR-10                       #
# Some part of the code was referenced from below                              #
# https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py   #
# ---------------------------------------------------------------------------- #



from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

tcmalloc: large alloc 1073750016 bytes == 0x5c702000 @  0x7faf7b2422a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [12]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyper-parameterr
batch_size = 200
num_epochs = 80
learning_rate = 0.03

# Image preprocessing modules
transform = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()])

# CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                             train=True, 
                                             transform=transform,
                                             download=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data/',
                                            train=False, 
                                            transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
shuffle=False)


# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
    
model = ResNet(ResidualBlock, [2, 2, 2, 2]).to(device)


# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

# Train the model
total_step = len(train_loader)
curr_lr = learning_rate
before = time.time()
log = []
for epoch in range(num_epochs):
    train_total = 0
    train_correct = 0
    train_loss = 0
    test_total = 0
    test_correct = 0
    test_loss = 0
    
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Accuracy calculation
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
        
        train_loss += loss.item() * labels.size(0)
    train_accuracy = train_correct / train_total

    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

    # Test the model
    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
    
    print(f'Epoch \t {epoch+1} \t {train_loss/train_total:.4f} \t {train_correct/train_total:.4f} \t {test_loss/test_total:.4f} \t {test_correct/test_total:.4f}')
    log.append([train_loss/train_total, train_correct/train_total, test_loss/test_total, test_correct/test_total])
print(f'elapsed time: {time.time() - before}')

cuda
Epoch 	 1 	 1.8917 	 0.2912 	 1.7312 	 0.3430
Epoch 	 2 	 1.5978 	 0.4082 	 1.6549 	 0.4008
Epoch 	 3 	 1.4059 	 0.4864 	 1.8376 	 0.3655
Epoch 	 4 	 1.2778 	 0.5367 	 1.5059 	 0.4806
Epoch 	 5 	 1.1783 	 0.5751 	 1.3339 	 0.5258
Epoch 	 6 	 1.0918 	 0.6081 	 1.2114 	 0.5676
Epoch 	 7 	 1.0268 	 0.6317 	 1.2839 	 0.5462
Epoch 	 8 	 0.9704 	 0.6499 	 1.1437 	 0.5891
Epoch 	 9 	 0.9258 	 0.6680 	 0.9807 	 0.6541
Epoch 	 10 	 0.8880 	 0.6831 	 1.0670 	 0.6230
Epoch 	 11 	 0.8474 	 0.6988 	 1.0158 	 0.6458
Epoch 	 12 	 0.8145 	 0.7102 	 1.0909 	 0.6296
Epoch 	 13 	 0.7908 	 0.7204 	 0.9497 	 0.6703
Epoch 	 14 	 0.7603 	 0.7331 	 0.9821 	 0.6667
Epoch 	 15 	 0.7400 	 0.7391 	 0.8199 	 0.7183
Epoch 	 16 	 0.7115 	 0.7503 	 0.7862 	 0.7220
Epoch 	 17 	 0.6921 	 0.7569 	 1.0372 	 0.6661
Epoch 	 18 	 0.6670 	 0.7656 	 0.9409 	 0.6771
Epoch 	 19 	 0.6490 	 0.7727 	 0.8596 	 0.7110
Epoch 	 20 	 0.6313 	 0.7771 	 0.8805 	 0.6959
Epoch 	 21 	 0.5696 	 0.8009 	 0.5922 	 0.7938
Epoch 	 22 	 0.55